# Comparison of different functions for fitting ridge regressions

In [1]:
import numpy as np
from scipy.linalg import svd
from sklearn.linear_model import Ridge
import sys
sys.path.append('../models')
from ridge_solvers import ridgesvd, get_beta


In [2]:
# Example usage:
T = 100
P = 1000
X = np.random.randn(T, P)
Y = np.random.randn(T)
lambd = np.array([1])

B_ridgesvd = ridgesvd(Y, X, lambd).flatten()
B_get_beta = get_beta(Y, X, lambd/T).flatten()
B_sklearn = np.linalg.inv(X.T @ X + lambd[0] * np.eye(P)) @ X.T @ Y
B_sklearn_svd = Ridge(alpha=lambd[0], fit_intercept=False, solver="svd").fit(X, Y).coef_
B_sklearn_auto = Ridge(alpha=lambd[0], fit_intercept=False, solver="auto").fit(X, Y).coef_
B_sklearn_cholesky = Ridge(alpha=lambd[0], fit_intercept=False, solver="cholesky").fit(X, Y).coef_


ref = B_ridgesvd
print(np.sum(abs(ref - B_get_beta)))
print(np.sum(abs(ref - B_ridgesvd)))
print(np.sum(abs(ref - B_sklearn)))
print(np.sum(abs(ref - B_sklearn_svd)))
print(np.sum(abs(ref - B_sklearn_auto)))
print(np.sum(abs(ref - B_sklearn_cholesky)))

3.2506332695025897e-14
0.0
9.43609697757336e-12
1.999355542237069e-14
1.6554899812115664e-14
1.6554899812115664e-14


In [4]:
%timeit ridgesvd(Y, X, lambd).flatten()
%timeit get_beta(Y, X, lambd/T).flatten()
%timeit np.linalg.inv(X.T @ X + lambd[0] * np.eye(P)) @ X.T @ Y
%timeit Ridge(alpha=lambd[0], fit_intercept=False, solver="svd").fit(X, Y).coef_
%timeit Ridge(alpha=lambd[0], fit_intercept=False, solver="auto").fit(X, Y).coef_
%timeit Ridge(alpha=lambd[0], fit_intercept=False, solver="cholesky").fit(X, Y).coef_


81.3 ms ± 32.7 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
20.8 ms ± 7.87 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
112 ms ± 4.77 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
42.8 ms ± 17.8 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
1.81 ms ± 33.9 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
1.8 ms ± 58.3 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


Standard deviations are relatively large for the first three functions, probably because something is being cached. 